In [1]:
# | default_exp nets/cait_3d

# Imports

In [2]:
# | export

import torch
from einops import repeat
from huggingface_hub import PyTorchModelHubMixin
from vision_architectures.utils.custom_base_model import CustomBaseModel
from torch import nn

from vision_architectures.layers.attention import Attention1D, Attention1DMLP

# Config

In [3]:
# | export


class CaiT3DStage1Config(CustomBaseModel):
    dim: int
    num_heads: int
    mlp_ratio: int
    layer_norm_eps: float
    attn_drop_prob: float = 0.0
    proj_drop_prob: float = 0.0
    mlp_drop_prob: float = 0.0

    stage1_depth: int


class CaiT3DStage2Config(CustomBaseModel):
    dim: int
    num_heads: int
    mlp_ratio: int
    layer_norm_eps: float
    attn_drop_prob: float = 0.0
    proj_drop_prob: float = 0.0
    mlp_drop_prob: float = 0.0

    stage2_depth: int

    num_class_tokens: int | None = None


class CaiT3DConfig(CaiT3DStage1Config, CaiT3DStage2Config):
    pass

# Architecture

### Basic Layers

In [4]:
# | export


class CaiT3DAttentionLayer(nn.Module):
    def __init__(
        self,
        dim,
        num_heads,
        mlp_ratio,
        layer_norm_eps,
        attn_drop_prob=0.0,
        proj_drop_prob=0.0,
        mlp_drop_prob=0.0,
    ):
        super().__init__()

        self.mhsa = Attention1D(dim, num_heads, attn_drop_prob=attn_drop_prob, proj_drop_prob=proj_drop_prob)
        self.gamma1 = nn.Parameter(torch.empty(1, 1, dim))
        self.layernorm1 = nn.LayerNorm(dim, eps=layer_norm_eps)
        self.mlp = Attention1DMLP(dim, mlp_ratio, "gelu", mlp_drop_prob)
        self.gamma2 = nn.Parameter(torch.empty(1, 1, dim))
        self.layernorm2 = nn.LayerNorm(dim, eps=layer_norm_eps)

        nn.init.uniform_(self.gamma1, a=-1e-4, b=1e-4)
        nn.init.uniform_(self.gamma2, a=-1e-4, b=1e-4)

    def forward(self, q: torch.Tensor, kv: torch.Tensor):
        # q: (b, num_tokens_in_q, dim)
        # kv: (b, num_tokens_in_kv, dim)

        res_connection1 = q
        # (b, num_tokens, dim)

        hidden_states = self.layernorm1(q)
        hidden_states = self.mhsa(hidden_states, kv, kv)
        hidden_states = self.gamma1 * hidden_states
        # (b, num_tokens, dim)

        res_connection2 = hidden_states + res_connection1
        # (b, num_tokens, dim)

        hidden_states = self.layernorm2(hidden_states)
        hidden_states = self.mlp(res_connection2)
        hidden_states = self.gamma2 * hidden_states
        # (b, num_tokens, dim)

        hidden_states = hidden_states + res_connection2
        # (b, num_tokens, dim)

        return hidden_states

In [5]:
test = CaiT3DAttentionLayer(52, 4, 2, 1e-6)
display(test)
display(test(torch.randn(2, 64, 52), torch.randn(2, 64, 52)).shape)


CaiT3DAttentionLayer(
  (mhsa): Attention1D(
    (W_q): Linear(in_features=52, out_features=52, bias=True)
    (W_k): Linear(in_features=52, out_features=52, bias=True)
    (W_v): Linear(in_features=52, out_features=52, bias=True)
    (proj): Linear(in_features=52, out_features=52, bias=True)
    (proj_drop): Dropout(p=0.0, inplace=False)
  )
  (layernorm1): LayerNorm((52,), eps=1e-06, elementwise_affine=True)
  (mlp): Attention1DMLP(
    (dense1): Linear(in_features=52, out_features=104, bias=True)
    (act): GELU(approximate='none')
    (dense2): Linear(in_features=104, out_features=52, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (layernorm2): LayerNorm((52,), eps=1e-06, elementwise_affine=True)
)

torch.Size([2, 64, 52])

In [6]:
# | export


class CaiT3DStage1Layer(CaiT3DAttentionLayer):  # Self attention without class tokens
    def __init__(
        self,
        dim,
        num_heads,
        mlp_ratio,
        layer_norm_eps,
        attn_drop_prob=0.0,
        proj_drop_prob=0.0,
        mlp_drop_prob=0.0,
    ):
        super().__init__(
            dim,
            num_heads,
            mlp_ratio,
            layer_norm_eps,
            attn_drop_prob=attn_drop_prob,
            proj_drop_prob=proj_drop_prob,
            mlp_drop_prob=mlp_drop_prob,
        )

    def forward(self, embeddings: torch.Tensor):
        # embeddings: (b, num_tokens, dim)
        return super().forward(embeddings, embeddings)

In [7]:
test = CaiT3DStage1Layer(52, 4, 2, 1e-6)
display(test)
display(test(torch.randn(2, 64, 52)).shape)


CaiT3DStage1Layer(
  (mhsa): Attention1D(
    (W_q): Linear(in_features=52, out_features=52, bias=True)
    (W_k): Linear(in_features=52, out_features=52, bias=True)
    (W_v): Linear(in_features=52, out_features=52, bias=True)
    (proj): Linear(in_features=52, out_features=52, bias=True)
    (proj_drop): Dropout(p=0.0, inplace=False)
  )
  (layernorm1): LayerNorm((52,), eps=1e-06, elementwise_affine=True)
  (mlp): Attention1DMLP(
    (dense1): Linear(in_features=52, out_features=104, bias=True)
    (act): GELU(approximate='none')
    (dense2): Linear(in_features=104, out_features=52, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (layernorm2): LayerNorm((52,), eps=1e-06, elementwise_affine=True)
)

torch.Size([2, 64, 52])

In [8]:
# | export


class CaiT3DStage2Layer(CaiT3DAttentionLayer):  # Attention with class tokens
    def __init__(
        self,
        dim,
        num_heads,
        mlp_ratio,
        layer_norm_eps,
        attn_drop_prob=0.0,
        proj_drop_prob=0.0,
        mlp_drop_prob=0.0,
    ):
        super().__init__(
            dim,
            num_heads,
            mlp_ratio,
            layer_norm_eps,
            attn_drop_prob=attn_drop_prob,
            proj_drop_prob=proj_drop_prob,
            mlp_drop_prob=mlp_drop_prob,
        )

    def forward(self, class_tokens: torch.Tensor, embeddings: torch.Tensor):
        # class_tokens: (b, num_class_tokens, dim)
        # embeddings: (b, num_embedding_tokens, dim)
        return super().forward(class_tokens, embeddings)

In [9]:
test = CaiT3DStage2Layer(52, 4, 2, 1e-6)
display(test)
display(test(torch.randn(2, 1, 52), torch.randn(2, 64, 52)).shape)


CaiT3DStage2Layer(
  (mhsa): Attention1D(
    (W_q): Linear(in_features=52, out_features=52, bias=True)
    (W_k): Linear(in_features=52, out_features=52, bias=True)
    (W_v): Linear(in_features=52, out_features=52, bias=True)
    (proj): Linear(in_features=52, out_features=52, bias=True)
    (proj_drop): Dropout(p=0.0, inplace=False)
  )
  (layernorm1): LayerNorm((52,), eps=1e-06, elementwise_affine=True)
  (mlp): Attention1DMLP(
    (dense1): Linear(in_features=52, out_features=104, bias=True)
    (act): GELU(approximate='none')
    (dense2): Linear(in_features=104, out_features=52, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (layernorm2): LayerNorm((52,), eps=1e-06, elementwise_affine=True)
)

torch.Size([2, 1, 52])

### Stages

In [10]:
# | export


class CaiT3DStage1(nn.Module, PyTorchModelHubMixin):
    def __init__(self, config: CaiT3DStage1Config):
        super().__init__()

        self.layers = nn.ModuleList(
            [
                CaiT3DStage1Layer(
                    config.dim,
                    config.num_heads,
                    config.mlp_ratio,
                    config.layer_norm_eps,
                    config.attn_drop_prob,
                    config.proj_drop_prob,
                    config.mlp_drop_prob,
                )
                for _ in range(config.stage1_depth)
            ]
        )

    def forward(self, embeddings: torch.Tensor):
        # embeddings: (b, num_tokens, dim)

        layer_outputs = []
        for layer in self.layers:
            embeddings = layer(embeddings)
            # (b, num_tokens, dim)

            layer_outputs.append(embeddings)

        return embeddings, layer_outputs

In [11]:
test_config = CaiT3DStage1Config.model_validate(
    {
        "dim": 54,
        "num_heads": 6,
        "mlp_ratio": 2,
        "layer_norm_eps": 1e-6,
        "attn_drop_prob": 0.0,
        "proj_drop_prob": 0.0,
        "mlp_drop_prob": 0.0,
        "stage1_depth": 5,
    }
)

test = CaiT3DStage1(test_config)
display(test)
o = test(torch.randn(2, 64, 54))
display((o[0].shape, [x.shape for x in o[1]]))


CaiT3DStage1(
  (layers): ModuleList(
    (0-4): 5 x CaiT3DStage1Layer(
      (mhsa): Attention1D(
        (W_q): Linear(in_features=54, out_features=54, bias=True)
        (W_k): Linear(in_features=54, out_features=54, bias=True)
        (W_v): Linear(in_features=54, out_features=54, bias=True)
        (proj): Linear(in_features=54, out_features=54, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (layernorm1): LayerNorm((54,), eps=1e-06, elementwise_affine=True)
      (mlp): Attention1DMLP(
        (dense1): Linear(in_features=54, out_features=108, bias=True)
        (act): GELU(approximate='none')
        (dense2): Linear(in_features=108, out_features=54, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (layernorm2): LayerNorm((54,), eps=1e-06, elementwise_affine=True)
    )
  )
)


(
    torch.Size([2, 64, 54]),
    [
        torch.Size([2, 64, 54]),
        torch.Size([2, 64, 54]),
        torch.Size([2, 64, 54]),
        torch.Size([2, 64, 54]),
        torch.Size([2, 64, 54])
    ]
)

In [12]:
# | export


class CaiT3DStage2(nn.Module, PyTorchModelHubMixin):
    def __init__(self, config: CaiT3DStage2Config):
        super().__init__()

        self.layers = nn.ModuleList(
            [
                CaiT3DStage2Layer(
                    config.dim,
                    config.num_heads,
                    config.mlp_ratio,
                    config.layer_norm_eps,
                    config.attn_drop_prob,
                    config.proj_drop_prob,
                    config.mlp_drop_prob,
                )
                for _ in range(config.stage2_depth)
            ]
        )

    def forward(self, class_tokens: torch.Tensor, embeddings: torch.Tensor):
        # class_tokens: (b, num_class_tokens, dim)
        # embeddings: (b, num_embed_tokens, dim)

        class_embeddings = class_tokens

        layer_outputs = []
        for layer in self.layers:
            class_embeddings = layer(class_embeddings, embeddings)
            # (b, num_class_tokens, dim)

            layer_outputs.append(class_embeddings)

        return class_embeddings, layer_outputs

In [13]:
test_config = CaiT3DStage2Config.model_validate({
    "dim": 54,
    "num_heads": 6,
    "mlp_ratio": 2,
    "layer_norm_eps": 1e-6,
    "attn_drop_prob": 0.0,
    "proj_drop_prob": 0.0,
    "mlp_drop_prob": 0.0,
    "stage2_depth": 5,
})

test = CaiT3DStage2(test_config)
display(test)
o = test(torch.randn(2, 1, 54), torch.randn(2, 64, 54))
display((o[0].shape, [x.shape for x in o[1]]))


CaiT3DStage2(
  (layers): ModuleList(
    (0-4): 5 x CaiT3DStage2Layer(
      (mhsa): Attention1D(
        (W_q): Linear(in_features=54, out_features=54, bias=True)
        (W_k): Linear(in_features=54, out_features=54, bias=True)
        (W_v): Linear(in_features=54, out_features=54, bias=True)
        (proj): Linear(in_features=54, out_features=54, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (layernorm1): LayerNorm((54,), eps=1e-06, elementwise_affine=True)
      (mlp): Attention1DMLP(
        (dense1): Linear(in_features=54, out_features=108, bias=True)
        (act): GELU(approximate='none')
        (dense2): Linear(in_features=108, out_features=54, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (layernorm2): LayerNorm((54,), eps=1e-06, elementwise_affine=True)
    )
  )
)


(
    torch.Size([2, 1, 54]),
    [
        torch.Size([2, 1, 54]),
        torch.Size([2, 1, 54]),
        torch.Size([2, 1, 54]),
        torch.Size([2, 1, 54]),
        torch.Size([2, 1, 54])
    ]
)

# Models

In [14]:
# | export


class CaiT3DStage2OnlyModel(nn.Module, PyTorchModelHubMixin):
    def __init__(self, config: CaiT3DStage2Config):
        super().__init__()

        self.num_class_tokens = config.num_class_tokens
        self.class_tokens = nn.Parameter(torch.randn(1, config.num_class_tokens, config.dim))

        self.class_attention = CaiT3DStage2(config)
        self.classifiers = nn.ModuleList([nn.Linear(config.dim, 1) for i in range(self.num_class_tokens)])

    def forward(self, embeddings: torch.Tensor):
        # embeddings: (b, num_embedding_tokens, dim)

        class_tokens = repeat(self.class_tokens, "1 n d -> b n d", b=embeddings.shape[0])
        # (b, num_class_tokens, dim)

        class_embeddings, layer_outputs = self.class_attention(class_tokens, embeddings)
        # class_embeddings: (b, num_class_tokens, dim)
        # layer_outputs: list of (b, num_embedding_tokens, dim)

        class_logits = torch.cat(
            [self.classifiers[i](class_embeddings[:, i]) for i in range(len(self.classifiers))], dim=1
        )
        # list of (b, num_classes) for each class token

        return class_logits, class_embeddings, layer_outputs

In [15]:
test_config = CaiT3DStage2Config.model_validate({
    "num_class_tokens": 3,
    "attn_drop_prob": 0.2,
    "dim": 768,
    "drop_prob": 0.2,
    "stage2_depth": 4,
    "mlp_ratio": 2,
    "layer_norm_eps": 1e-6,
    "mlp_drop_prob": 0.2,
    "num_heads": 4,
    "proj_drop_prob": 0.2,
})

test = CaiT3DStage2OnlyModel(test_config)
display(test)
o = test(
    torch.randn(2, 4096, 768),
)
display((o[0].shape, o[1].shape, [x.shape for x in o[2]]))


CaiT3DStage2OnlyModel(
  (class_attention): CaiT3DStage2(
    (layers): ModuleList(
      (0-3): 4 x CaiT3DStage2Layer(
        (mhsa): Attention1D(
          (W_q): Linear(in_features=768, out_features=768, bias=True)
          (W_k): Linear(in_features=768, out_features=768, bias=True)
          (W_v): Linear(in_features=768, out_features=768, bias=True)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.2, inplace=False)
        )
        (layernorm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Attention1DMLP(
          (dense1): Linear(in_features=768, out_features=1536, bias=True)
          (act): GELU(approximate='none')
          (dense2): Linear(in_features=1536, out_features=768, bias=True)
          (dropout): Dropout(p=0.2, inplace=False)
        )
        (layernorm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      )
    )
  )
  (classifiers): ModuleList(
    (0-2): 3 x Linear(in_


(
    torch.Size([2, 3]),
    torch.Size([2, 3, 768]),
    [
        torch.Size([2, 3, 768]),
        torch.Size([2, 3, 768]),
        torch.Size([2, 3, 768]),
        torch.Size([2, 3, 768])
    ]
)

In [16]:
# | export


class CaiT3DModel(nn.Module, PyTorchModelHubMixin):
    def __init__(self, config: CaiT3DConfig):
        super().__init__()

        self.num_class_tokens = config.num_class_tokens
        self.class_tokens = nn.Parameter(torch.randn(1, config.num_class_tokens, config.dim))

        self.self_attention = CaiT3DStage1(config)
        self.class_attention = CaiT3DStage2(config)
        self.classifiers = nn.ModuleList([nn.Linear(config.dim, 1) for i in range(self.num_class_tokens)])

    def forward(self, tokens: torch.Tensor):
        # tokens: (b, num_embedding_tokens, dim)

        embeddings, layer_outputs1 = self.self_attention(tokens)

        class_tokens = repeat(self.class_tokens, "1 n d -> b n d", b=embeddings.shape[0])
        # (b, num_class_tokens, dim)

        class_embeddings, layer_outputs2 = self.class_attention(class_tokens, embeddings)
        # class_embeddings: (b, num_class_tokens, dim)
        # layer_outputs: list of (b, num_embedding_tokens, dim)

        class_logits = torch.cat(
            [self.classifiers[i](class_embeddings[:, i]) for i in range(len(self.classifiers))], dim=1
        )
        # list of (b, num_classes) for each class token

        return class_logits, class_embeddings, [layer_outputs1, layer_outputs2]

In [17]:
test_config = CaiT3DConfig.model_validate({
    "num_class_tokens": 3,
    "attn_drop_prob": 0.2,
    "dim": 768,
    "drop_prob": 0.2,
    "stage1_depth": 2,
    "stage2_depth": 2,
    "mlp_ratio": 2,
    "layer_norm_eps": 1e-6,
    "mlp_drop_prob": 0.2,
    "num_heads": 4,
    "proj_drop_prob": 0.2,
})

test = CaiT3DModel(test_config)
display(test)
o = test(
    torch.randn(2, 4096, 768),
)
display((o[0].shape, o[1].shape, [[x.shape for x in o[2][0]], [x.shape for x in o[2][1]]]))


CaiT3DModel(
  (self_attention): CaiT3DStage1(
    (layers): ModuleList(
      (0-1): 2 x CaiT3DStage1Layer(
        (mhsa): Attention1D(
          (W_q): Linear(in_features=768, out_features=768, bias=True)
          (W_k): Linear(in_features=768, out_features=768, bias=True)
          (W_v): Linear(in_features=768, out_features=768, bias=True)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.2, inplace=False)
        )
        (layernorm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Attention1DMLP(
          (dense1): Linear(in_features=768, out_features=1536, bias=True)
          (act): GELU(approximate='none')
          (dense2): Linear(in_features=1536, out_features=768, bias=True)
          (dropout): Dropout(p=0.2, inplace=False)
        )
        (layernorm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      )
    )
  )
  (class_attention): CaiT3DStage2(
    (layers): ModuleList(
    


(
    torch.Size([2, 3]),
    torch.Size([2, 3, 768]),
    [
        [torch.Size([2, 4096, 768]), torch.Size([2, 4096, 768])],
        [torch.Size([2, 3, 768]), torch.Size([2, 3, 768])]
    ]
)

# nbdev

In [18]:
!nbdev_export